In [34]:
import os, csv, json, bz2

from split_articles_to_csv import split_articles_to_csv
from coproximity_create_vocabulary.data_conf import base_vocab_folder
from ade_imi.data_conf import base_data_folder

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [35]:
old_vocab_folder = base_data_folder + 'whole/vocabulary/wiki_title_fr/ngram_title_wiki/wiki_title_best_100000/'
multiple_synonyms_file = old_vocab_folder + 'multiple_synonyms.json'
with open(multiple_synonyms_file, encoding='utf8') as f :
    multiple_synonyms = json.load(f)

In [36]:
set_titles = {title.replace('_', ' ') for _, titles in multiple_synonyms for title in titles}


In [36]:
set_titles = {'Côte-d\'Or', 'Cuisine', 'Catholicisme', 'Informatique', 'Iran'}

In [37]:
r'E:\UNIL\backend\data\wikipedia\whole'.replace('\\', '/')

'E:/UNIL/backend/data/wikipedia/whole'

In [ ]:
project = 'fr'
#from_xml_bz2 = base_vocab_folder + f'/french/dumps/wiki_{project}_dump.xml.bz2'
from_xml_bz2 = 'E:/UNIL/backend/data/wikipedia/whole/wiki_fr_dump.xml.bz2'
res = split_articles_to_csv (
    whole_dir = '',
    from_xml_bz2 = from_xml_bz2,
    dump_save_to = None ,
    threshold_skip_little = 100,
    what_to_do_result = 'return_dict_title',
    get_only_ids = None,
    get_only_title = set_titles,
)

In [ ]:
set_titles

In [45]:
len(res), len(set_titles) - len([idx for idx in set_titles if '_' in idx])

(161, 10)

In [46]:
[idx for idx in set_titles if not '_' in idx]

['Indestructibles',
 'Mutualisme',
 'Sceaux',
 'Réflexivité',
 'SNK',
 'Éternels',
 'Q-Tip',
 'Parisiennes',
 'Soninké',
 'Gardiennes']

In [43]:
set_titles

{'11e_arrondissement_de_Paris',
 '12e_arrondissement_de_Paris',
 '13e_arrondissement_de_Paris',
 'Académie_militaire_de_Saint-Cyr_Coëtquidan',
 'Accès_à_Internet',
 'Adam_et_Ève',
 'Administration_aux_États-Unis',
 'Affaires_du_voile_islamique_en_France',
 'Agence_française_de_développement',
 'Agression_sexuelle',
 'Alces_americanus',
 "Allemagne_de_l'Ouest",
 "Alternative_pour_l'Allemagne",
 'Amour,_Gloire_et_Beauté',
 'Anti-inflammatoire_non_stéroïdien',
 'Appareil_cardiovasculaire',
 'Arabie_saoudite',
 'Arbre_de_vie',
 'Architecture_distribuée',
 'Architecture_romane',
 'Arme_nucléaire',
 'Art_numérique',
 'Art_roman',
 "Assassin's_Creed",
 "Assassin's_Creed_Unity",
 'Autobus_de_Lyon',
 'Axiome_des_parallèles',
 'Bailliage_et_sénéchaussée',
 'Belles_Familles',
 'Blood_Diamond',
 'Bombe_A',
 'Brigitte_Bardot',
 'Cahier_des_charges',
 'Calcul_distribué',
 'Camp_romain',
 'Cannabis_indica',
 'Capteur_solaire_photovoltaïque',
 'Carence_nutritionnelle',
 'Centrale_à_inertie',
 "Changem

In [39]:
index_filename = base_vocab_folder + 'french/dumps/frwiki-latest-pages-articles-multistream-index.txt.bz2'
def get_dict_search_index(index_filename):
    data_length = start_byte = 0
    curr_res, res = set(), {}
    
    with bz2.open(index_filename) as f_in  :
        for line in f_in :
            curr_start_byte, _, *title = line.decode('utf8').strip().split(':')
            title = ':'.join(title)
            curr_start_byte = int(curr_start_byte)
            if curr_start_byte != start_byte :
                if curr_res :
                    res.update({title: (start_byte, curr_start_byte - start_byte) for title in curr_res})
                curr_res = set()
                start_byte = curr_start_byte
                
            curr_res.add(title)
            
    if curr_res :
        res.update({title: (start_byte, curr_start_byte - start_byte) for title in curr_res})
    return res


dict_search_index = get_dict_search_index(index_filename)

In [41]:
wiki_filename = base_vocab_folder + 'french/dumps/wiki_fr_dump.xml.bz2'
temp_filename = base_vocab_folder + 'french/dumps/temp_wiki_fr_dump.xml.bz2'
def get_title_from_chunk(wiki_filename, temp_filename, title):
    
    if not title in dict_search_index:
        return
    
    start_byte, data_length = dict_search_index[title]

    with open(wiki_filename, 'rb') as wiki_file:
        wiki_file.seek(start_byte)
        data = wiki_file.read(data_length)

    with open(temp_filename, 'wb') as temp_file:
        temp_file.write(data)

    return split_articles_to_csv (
        whole_dir = '',
        from_xml_bz2 = temp_filename,
        dump_save_to = None ,
        threshold_skip_little = 0,
        what_to_do_result = 'return_dict_title',
        get_only_ids = None,
        get_only_title = {title},
        is_printing=False,
    ).get(title)

res = {title: get_title_from_chunk(wiki_filename, temp_filename, title) for title in set_titles}

In [ ]:
{idx: elem[:30] for idx, elem in res.items() if elem}